In [1]:
import numpy as np
import random as rng
import typing

In [6]:
# mutation functions

def bitFlip(population, flip_probability):
    p = flip_probability
    
    for i in range(len(population)):
        if p >= np.random.rand(1, 1):
            population[i] = not(population[i])
    return population
#end

# gaussian convolution
def gaussianConvolution(population, variance, noise_probability=1):
    p = noise_probability
    var = variance        #variance of the normal distribution
    mu = 0                #mean of the normal distribution
    min = 0.0
    max = 100.0
    
    for i in range(len(population)):
        randy_num = rng.normalvariate(mu, var)
        
        if p >= np.random.rand(1, 1):
            while not (min <= population[i] + randy_num <= max):
                randy_num = rng.normalvariate(mu, var)
            #end
            population[i] = (population[i] + randy_num)
        #end
    return population
#end

mutate_functions = {"bit flip":bitFlip, "g convolve":gaussianConvolution}

array([2.0767737 , 2.        , 2.        , 3.50329258, 1.        ,
       9.66338656, 6.51394299, 6.53594322])

In [8]:
# test mutation

pop = np.array([2, 2, 2, 2, 1, 1, 1, 1], dtype="float")
mut_pop = mutate_functions["g convolve"](pop, 5, 0.5)
mut_pop

array([18.38784595,  4.69066395,  5.25475214,  2.        ,  5.28077253,
        2.46121306,  2.40320841,  1.        ])

In [104]:
# recombination

def onePtCrossover(vec_1, vec_2):
    #TODO handle unequal-length vecs
    #TODO handle empty vecs
    
    # choose a random integer (between bidx and pop length)
    cross_beg_idx = int(abs(rng.uniform(0, len(vec_1))))
    
    # as lonng as the selected beginning of crossover is not the first index...
    # (not sure why this is. I'd think this would break evolution)
    if cross_beg_idx != 0:
        for idx in range(cross_beg_idx, len(vec_1)):
            swap = vec_1[idx]
            vec_1[idx] = vec_2[idx]
            vec_2[idx] = swap
        #end
    #end
    return vec_1, vec_2
#end

recombine_fns = {"1-pt-crossover":onePtCrossover}

In [103]:
# test recombination
pop_1 = [4.0, 3.2, 1.0, 0.5, 6.2]
pop_2 = [10.0, 15.0, 12.2, 9.5, 5.0]

pop_3, pop_4 = onePtCrossover(pop_1, pop_2)

print(pop_3)
pop_4

[4.0, 3.2, 12.2, 9.5, 5.0]


[10.0, 15.0, 1.0, 0.5, 6.2]

In [ ]:
#selection

def tournamentSelect(population, tournament_size):
    t = tournament_size
    
    select_idx = rng.uniform(0, len(population))
    best = population[select_idx]
    # for every population element between second and "tournament size"...
    for i in range(1, len(population)):
        # choose a "next" at random from pop
        select_idx = rng.uniform(0, len(population))
        next = population[select_idx]
        # if fitness of Next better than best...
        if fitness(next) > fitness(best)
            # best = next
            best = next
        #end
    return best
#end